# Training

In this notebook, the selected models are trained on the datasets generated by the coreset selectors.

### Models

- Resnet18
- Mobilenet
- Inception
- VGGnet

### Training Epoch Function

Function that trains and evaluates the model for a given number of epochs.
Switches from optimizer1 to optimizer2 after 10 epochs, for using SWAT technique

[https://arxiv.org/pdf/1712.07628.pdf](https://arxiv.org/pdf/1712.07628.pdf)

In [1]:
from datetime import datetime
from sklearn.metrics import accuracy_score
import torch

def train_model(epochs, net, net_name, criterion, optimizer, train_loader, val_loader,scheduler=None, verbose=True, device='cuda'):
    freq = max(epochs//20,1)
    
    accuracies = []
 
    for epoch in range(1, epochs+1):
        net.train()

        losses_train = []
        for X, target in train_loader:
            X, target = X.to(device), target.to(device)
            
            net_output = net(X)
            loss = criterion(net_output, target)
            losses_train.append(float(loss))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            #scheduler is cosine annealing, so its called in the step loop
            if scheduler != None:
                scheduler.step()
        
        if verbose and epoch%freq==0:

            y_pred_val =  []
            y_true_val = []

            net.eval()

            losses_val = []
            for X, target in val_loader:
                X, target = X.to(device), target.to(device)

                # Compute the validation loss
                target_hat_val = net(X)

                loss = criterion(target_hat_val, target)
                losses_val.append(float(loss))

                y_pred_val.extend(target_hat_val.argmax(1).tolist())
                y_true_val.extend(target.tolist())

            mean_val = sum(losses_val)/len(losses_val)
            mean_train = sum(losses_train)/len(losses_train)
            
            accuracies.append(accuracy_score(y_true_val, y_pred_val))
            
            print('Timestamp: ', datetime.now().strftime("%H:%M:%S"), \
                  '\tVal epoch {}'.format(epoch), \
                  '\n\tModel: {}'.format(net_name), \
                  '\n\tLoss Train: {:.3}'.format(mean_train), \
                  ',\n\tLoss Test: {:.3}'.format(mean_val),\
                  ',\n\tAccuracy on test: {:.3}'.format(accuracy_score(y_true_val, y_pred_val)) )
            
    return accuracies

### Training Evaluation Function

Used to evaluate model after training. Function puts the result in a log

### Dataset

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms

# fix random seed
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True

#factors selected from torch docs
mean = (0.4914, 0.4822, 0.4465)
std = (0.2471, 0.2435, 0.2616)


#preprocessing
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data/',
                                            train=False, 
                                            transform=transform,
                                            download=True)

0it [00:00, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data/
Files already downloaded and verified


### Coreset Training Function

Function that trains the model on selected datapoints from the dataset. The argument is given as a list of indices, to make it suitable for core-set selection techniques

In [4]:
import sys
sys.path.append('../submodules/PyTorch_CIFAR10')

import numpy as np
import torch.nn as nn
from schduler import WarmupCosineLR

def train_and_save_models(models, model_names, train_indices, percentage_of_dataset, selector_name=None):
    
    train_datasubset = [train_dataset[i] for i in train_indices]
    
    # Data loader
    train_loader = torch.utils.data.DataLoader(dataset=train_datasubset,
                                               batch_size=128, 
                                               shuffle=True,
                                               drop_last=True
                                              )

    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                              batch_size=128, 
                                              shuffle=False,
                                              drop_last=True
                                             )
    
    # Device configuration
    device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
    
    # Training parameters
    num_epochs = 100
    learning_rate = 1e-2
    weight_decay = 1e-2
    total_steps = num_epochs * len(train_loader)
    
    #train selected models on subset
    for model, label in zip(models, model_names):

        model = model.to(device)

        # Loss and optimizer
        criterion = nn.CrossEntropyLoss()
        
        optimizer = torch.optim.SGD(
            model.parameters(),
            lr=learning_rate,
            weight_decay=weight_decay,
            momentum=0.9,
            nesterov=True
        )
        
        # Scheduler
        scheduler = WarmupCosineLR(
            optimizer,
            warmup_epochs=total_steps * 0.3,
            max_epochs = total_steps)

        # Train the model
        accuracies = train_model(num_epochs, model, label, criterion, optimizer, train_loader, test_loader, scheduler=scheduler, verbose=True, device=device)
        
        # take model as parameter, not cacluclate it
        # percentage_of_dataset = np.round((len(train_datasubset)/len(train_dataset))*100).astype(int)

        if selector_name == None:
            weights_filename = 'model_weights/{:03}_{}.pt'.format(percentage_of_dataset, label)
            results_filename = 'accuracy_results/{:03}_{}.csv'.format(percentage_of_dataset, label)
        else:
            weights_filename = 'model_weights/{:03}_{}_{}.pt'.format(percentage_of_dataset, label, selector_name)
            results_filename = 'accuracy_results/{:03}_{}_{}.csv'.format(percentage_of_dataset, label, selector_name)
        torch.save(model.state_dict(), weights_filename)
        
        np.savetxt(results_filename, accuracies, delimiter=',')
        

In [3]:
from training_utilities import train_and_save_models

### Create Models

In [6]:
import sys
sys.path.append('../../../submodules/PyTorch_CIFAR10/cifar10_models/')

from resnet import resnet18
from mobilenetv2 import mobilenet_v2
from densenet import densenet121
from vgg import vgg11_bn

# Models
model_resnet = resnet18()
model_mobilenet = mobilenet_v2()
model_vgg = vgg11_bn()
model_densenet = densenet121()

models = [model_resnet, model_mobilenet, model_vgg, model_densenet]
model_names = ['resnet', 'mobilenet', 'vgg', 'densenet']

### Train on "full" dataset: 20000 datapoints

In [2]:
import sys
sys.path.append('../../../dataset/')
import dataset_manager

train_indices = dataset_manager.CIFAR10_subset_indices()

train_and_save_models(models, model_names, train_indices, 100)

### Train for data subsets

#### GLISTER

In [29]:
import numpy as np
sys.path.append('../../Glister/GlisterImage/indices/')

indices_50_glister = np.loadtxt('../../Glister/GlisterImage/indices/glister_indices_50.csv', delimiter=',').astype(int)
indices_30_glister = np.loadtxt('../../Glister/GlisterImage/indices/glister_indices_30.csv', delimiter=',').astype(int)
indices_10_glister = np.loadtxt('../../Glister/GlisterImage/indices/glister_indices_10.csv', delimiter=',').astype(int)

train_and_save_models(models, model_names, indices_50_glister, 50, selector_name='glister')
train_and_save_models(models, model_names, indices_30_glister, 30, selector_name='glister')
train_and_save_models(models, model_names, indices_10_glister, 10, selector_name='glister')


Timestamp:  08:26:50 	Val epoch 5 
	Model: resnet 
	Loss Train: 1.58 ,
	Loss Test: 1.65 ,
	Accuracy on test: 0.39
Timestamp:  08:27:26 	Val epoch 10 
	Model: resnet 
	Loss Train: 0.388 ,
	Loss Test: 1.76 ,
	Accuracy on test: 0.421
Timestamp:  08:28:02 	Val epoch 15 
	Model: resnet 
	Loss Train: 0.616 ,
	Loss Test: 2.04 ,
	Accuracy on test: 0.415
Timestamp:  08:28:39 	Val epoch 20 
	Model: resnet 
	Loss Train: 0.338 ,
	Loss Test: 2.11 ,
	Accuracy on test: 0.463
Timestamp:  08:29:15 	Val epoch 25 
	Model: resnet 
	Loss Train: 0.217 ,
	Loss Test: 2.06 ,
	Accuracy on test: 0.458
Timestamp:  08:29:52 	Val epoch 30 
	Model: resnet 
	Loss Train: 0.277 ,
	Loss Test: 1.94 ,
	Accuracy on test: 0.473
Timestamp:  08:30:29 	Val epoch 35 
	Model: resnet 
	Loss Train: 0.247 ,
	Loss Test: 1.76 ,
	Accuracy on test: 0.497
Timestamp:  08:31:05 	Val epoch 40 
	Model: resnet 
	Loss Train: 0.19 ,
	Loss Test: 1.79 ,
	Accuracy on test: 0.503
Timestamp:  08:31:41 	Val epoch 45 
	Model: resnet 
	Loss Train: 0.1

Timestamp:  09:30:38 	Val epoch 55 
	Model: densenet 
	Loss Train: 0.231 ,
	Loss Test: 1.16 ,
	Accuracy on test: 0.661
Timestamp:  09:31:55 	Val epoch 60 
	Model: densenet 
	Loss Train: 0.189 ,
	Loss Test: 1.26 ,
	Accuracy on test: 0.642
Timestamp:  09:33:12 	Val epoch 65 
	Model: densenet 
	Loss Train: 0.143 ,
	Loss Test: 1.18 ,
	Accuracy on test: 0.674
Timestamp:  09:34:29 	Val epoch 70 
	Model: densenet 
	Loss Train: 0.102 ,
	Loss Test: 1.07 ,
	Accuracy on test: 0.704
Timestamp:  09:35:47 	Val epoch 75 
	Model: densenet 
	Loss Train: 0.0633 ,
	Loss Test: 1.06 ,
	Accuracy on test: 0.715
Timestamp:  09:37:04 	Val epoch 80 
	Model: densenet 
	Loss Train: 0.0393 ,
	Loss Test: 0.945 ,
	Accuracy on test: 0.752
Timestamp:  09:38:22 	Val epoch 85 
	Model: densenet 
	Loss Train: 0.0355 ,
	Loss Test: 0.871 ,
	Accuracy on test: 0.77
Timestamp:  09:39:40 	Val epoch 90 
	Model: densenet 
	Loss Train: 0.0355 ,
	Loss Test: 0.882 ,
	Accuracy on test: 0.766
Timestamp:  09:40:58 	Val epoch 95 
	Model

Timestamp:  10:14:38 	Val epoch 5 
	Model: densenet 
	Loss Train: 0.276 ,
	Loss Test: 0.939 ,
	Accuracy on test: 0.733
Timestamp:  10:15:27 	Val epoch 10 
	Model: densenet 
	Loss Train: 0.299 ,
	Loss Test: 1.33 ,
	Accuracy on test: 0.627
Timestamp:  10:16:16 	Val epoch 15 
	Model: densenet 
	Loss Train: 0.29 ,
	Loss Test: 1.23 ,
	Accuracy on test: 0.635
Timestamp:  10:17:05 	Val epoch 20 
	Model: densenet 
	Loss Train: 0.284 ,
	Loss Test: 1.33 ,
	Accuracy on test: 0.629
Timestamp:  10:17:54 	Val epoch 25 
	Model: densenet 
	Loss Train: 0.258 ,
	Loss Test: 1.34 ,
	Accuracy on test: 0.613
Timestamp:  10:18:42 	Val epoch 30 
	Model: densenet 
	Loss Train: 0.246 ,
	Loss Test: 1.52 ,
	Accuracy on test: 0.579
Timestamp:  10:19:31 	Val epoch 35 
	Model: densenet 
	Loss Train: 0.205 ,
	Loss Test: 1.19 ,
	Accuracy on test: 0.651
Timestamp:  10:20:20 	Val epoch 40 
	Model: densenet 
	Loss Train: 0.168 ,
	Loss Test: 1.25 ,
	Accuracy on test: 0.646
Timestamp:  10:21:08 	Val epoch 45 
	Model: dense

Timestamp:  10:42:13 	Val epoch 55 
	Model: vgg 
	Loss Train: 0.0143 ,
	Loss Test: 1.61 ,
	Accuracy on test: 0.539
Timestamp:  10:42:23 	Val epoch 60 
	Model: vgg 
	Loss Train: 0.0157 ,
	Loss Test: 1.53 ,
	Accuracy on test: 0.549
Timestamp:  10:42:32 	Val epoch 65 
	Model: vgg 
	Loss Train: 0.0163 ,
	Loss Test: 1.55 ,
	Accuracy on test: 0.55
Timestamp:  10:42:42 	Val epoch 70 
	Model: vgg 
	Loss Train: 0.017 ,
	Loss Test: 1.53 ,
	Accuracy on test: 0.544
Timestamp:  10:42:51 	Val epoch 75 
	Model: vgg 
	Loss Train: 0.0171 ,
	Loss Test: 1.52 ,
	Accuracy on test: 0.551
Timestamp:  10:43:01 	Val epoch 80 
	Model: vgg 
	Loss Train: 0.017 ,
	Loss Test: 1.53 ,
	Accuracy on test: 0.545
Timestamp:  10:43:10 	Val epoch 85 
	Model: vgg 
	Loss Train: 0.0169 ,
	Loss Test: 1.53 ,
	Accuracy on test: 0.544
Timestamp:  10:43:20 	Val epoch 90 
	Model: vgg 
	Loss Train: 0.0168 ,
	Loss Test: 1.51 ,
	Accuracy on test: 0.55
Timestamp:  10:43:30 	Val epoch 95 
	Model: vgg 
	Loss Train: 0.0172 ,
	Loss Test: 1

#### K-Centers

In [6]:
import numpy as np
sys.path.append('../../greedy_k_centers/')

indices_50_k_centers = np.loadtxt('../../greedy_k_centers/k_centers_indices.csv', delimiter=',').astype(int)
indices_30_k_centers = np.loadtxt('../../greedy_k_centers/k_centers_indices.csv', delimiter=',')[:6000].astype(int)
indices_10_k_centers = np.loadtxt('../../greedy_k_centers/k_centers_indices.csv', delimiter=',')[:2000].astype(int)

train_and_save_models(models, model_names, indices_50_k_centers, 50, selector_name='k-centers')
train_and_save_models(models, model_names, indices_30_k_centers, 30, selector_name='k-centers')
train_and_save_models(models, model_names, indices_10_k_centers, 10, selector_name='k-centers')

Timestamp:  10:58:37 	Val epoch 5 
	Model: resnet 
	Loss Train: 1.57 ,
	Loss Test: 1.64 ,
	Accuracy on test: 0.389
Timestamp:  10:59:13 	Val epoch 10 
	Model: resnet 
	Loss Train: 0.376 ,
	Loss Test: 1.77 ,
	Accuracy on test: 0.438
Timestamp:  10:59:50 	Val epoch 15 
	Model: resnet 
	Loss Train: 0.52 ,
	Loss Test: 1.93 ,
	Accuracy on test: 0.464
Timestamp:  11:00:26 	Val epoch 20 
	Model: resnet 
	Loss Train: 0.287 ,
	Loss Test: 2.1 ,
	Accuracy on test: 0.452
Timestamp:  11:01:03 	Val epoch 25 
	Model: resnet 
	Loss Train: 0.272 ,
	Loss Test: 1.93 ,
	Accuracy on test: 0.479
Timestamp:  11:01:39 	Val epoch 30 
	Model: resnet 
	Loss Train: 0.26 ,
	Loss Test: 1.77 ,
	Accuracy on test: 0.488
Timestamp:  11:02:15 	Val epoch 35 
	Model: resnet 
	Loss Train: 0.212 ,
	Loss Test: 1.75 ,
	Accuracy on test: 0.501
Timestamp:  11:02:52 	Val epoch 40 
	Model: resnet 
	Loss Train: 0.182 ,
	Loss Test: 1.7 ,
	Accuracy on test: 0.517
Timestamp:  11:03:28 	Val epoch 45 
	Model: resnet 
	Loss Train: 0.16 

Timestamp:  12:02:09 	Val epoch 55 
	Model: densenet 
	Loss Train: 0.214 ,
	Loss Test: 1.17 ,
	Accuracy on test: 0.657
Timestamp:  12:03:26 	Val epoch 60 
	Model: densenet 
	Loss Train: 0.184 ,
	Loss Test: 1.12 ,
	Accuracy on test: 0.679
Timestamp:  12:04:43 	Val epoch 65 
	Model: densenet 
	Loss Train: 0.15 ,
	Loss Test: 1.26 ,
	Accuracy on test: 0.65
Timestamp:  12:06:00 	Val epoch 70 
	Model: densenet 
	Loss Train: 0.0954 ,
	Loss Test: 1.17 ,
	Accuracy on test: 0.677
Timestamp:  12:07:17 	Val epoch 75 
	Model: densenet 
	Loss Train: 0.0799 ,
	Loss Test: 1.07 ,
	Accuracy on test: 0.713
Timestamp:  12:08:35 	Val epoch 80 
	Model: densenet 
	Loss Train: 0.0393 ,
	Loss Test: 0.978 ,
	Accuracy on test: 0.742
Timestamp:  12:09:52 	Val epoch 85 
	Model: densenet 
	Loss Train: 0.0358 ,
	Loss Test: 0.895 ,
	Accuracy on test: 0.763
Timestamp:  12:11:09 	Val epoch 90 
	Model: densenet 
	Loss Train: 0.0347 ,
	Loss Test: 0.909 ,
	Accuracy on test: 0.759
Timestamp:  12:12:27 	Val epoch 95 
	Model

Timestamp:  12:46:01 	Val epoch 5 
	Model: densenet 
	Loss Train: 0.247 ,
	Loss Test: 1.17 ,
	Accuracy on test: 0.681
Timestamp:  12:46:49 	Val epoch 10 
	Model: densenet 
	Loss Train: 0.199 ,
	Loss Test: 1.12 ,
	Accuracy on test: 0.687
Timestamp:  12:47:38 	Val epoch 15 
	Model: densenet 
	Loss Train: 0.273 ,
	Loss Test: 1.31 ,
	Accuracy on test: 0.631
Timestamp:  12:48:27 	Val epoch 20 
	Model: densenet 
	Loss Train: 0.228 ,
	Loss Test: 1.31 ,
	Accuracy on test: 0.628
Timestamp:  12:49:15 	Val epoch 25 
	Model: densenet 
	Loss Train: 0.257 ,
	Loss Test: 1.5 ,
	Accuracy on test: 0.57
Timestamp:  12:50:04 	Val epoch 30 
	Model: densenet 
	Loss Train: 0.255 ,
	Loss Test: 1.2 ,
	Accuracy on test: 0.652
Timestamp:  12:50:53 	Val epoch 35 
	Model: densenet 
	Loss Train: 0.206 ,
	Loss Test: 1.44 ,
	Accuracy on test: 0.606
Timestamp:  12:51:41 	Val epoch 40 
	Model: densenet 
	Loss Train: 0.191 ,
	Loss Test: 1.2 ,
	Accuracy on test: 0.653
Timestamp:  12:52:30 	Val epoch 45 
	Model: densenet 

Timestamp:  13:13:36 	Val epoch 55 
	Model: vgg 
	Loss Train: 0.0169 ,
	Loss Test: 1.3 ,
	Accuracy on test: 0.582
Timestamp:  13:13:46 	Val epoch 60 
	Model: vgg 
	Loss Train: 0.0176 ,
	Loss Test: 1.29 ,
	Accuracy on test: 0.583
Timestamp:  13:13:55 	Val epoch 65 
	Model: vgg 
	Loss Train: 0.017 ,
	Loss Test: 1.28 ,
	Accuracy on test: 0.589
Timestamp:  13:14:05 	Val epoch 70 
	Model: vgg 
	Loss Train: 0.0173 ,
	Loss Test: 1.27 ,
	Accuracy on test: 0.591
Timestamp:  13:14:14 	Val epoch 75 
	Model: vgg 
	Loss Train: 0.0173 ,
	Loss Test: 1.3 ,
	Accuracy on test: 0.583
Timestamp:  13:14:24 	Val epoch 80 
	Model: vgg 
	Loss Train: 0.0178 ,
	Loss Test: 1.28 ,
	Accuracy on test: 0.581
Timestamp:  13:14:33 	Val epoch 85 
	Model: vgg 
	Loss Train: 0.017 ,
	Loss Test: 1.27 ,
	Accuracy on test: 0.592
Timestamp:  13:14:43 	Val epoch 90 
	Model: vgg 
	Loss Train: 0.0175 ,
	Loss Test: 1.29 ,
	Accuracy on test: 0.585
Timestamp:  13:14:52 	Val epoch 95 
	Model: vgg 
	Loss Train: 0.0173 ,
	Loss Test: 1

#### Random

In [5]:
# get initial dataset
train_indices = dataset_manager.CIFAR10_subset_indices()

indices_50_random = np.random.choice(train_indices, size=10000, replace=False)
indices_30_random = np.random.choice(train_indices, size=6000, replace=False)
indices_10_random = np.random.choice(train_indices, size=2000, replace=False)

train_and_save_models(models, model_names, indices_50_random, 50, selector_name='random')
train_and_save_models(models, model_names, indices_30_random, 30, selector_name='random')
train_and_save_models(models, model_names, indices_10_random, 10, selector_name='random')

Timestamp:  13:24:48 	Val epoch 5 
	Model: resnet 
	Loss Train: 1.56 ,
	Loss Test: 1.67 ,
	Accuracy on test: 0.383
Timestamp:  13:25:24 	Val epoch 10 
	Model: resnet 
	Loss Train: 0.361 ,
	Loss Test: 1.8 ,
	Accuracy on test: 0.43
Timestamp:  13:26:00 	Val epoch 15 
	Model: resnet 
	Loss Train: 0.481 ,
	Loss Test: 1.87 ,
	Accuracy on test: 0.45
Timestamp:  13:26:37 	Val epoch 20 
	Model: resnet 
	Loss Train: 0.358 ,
	Loss Test: 1.94 ,
	Accuracy on test: 0.465
Timestamp:  13:27:13 	Val epoch 25 
	Model: resnet 
	Loss Train: 0.313 ,
	Loss Test: 2.12 ,
	Accuracy on test: 0.453
Timestamp:  13:27:49 	Val epoch 30 
	Model: resnet 
	Loss Train: 0.293 ,
	Loss Test: 1.97 ,
	Accuracy on test: 0.458
Timestamp:  13:28:26 	Val epoch 35 
	Model: resnet 
	Loss Train: 0.221 ,
	Loss Test: 1.95 ,
	Accuracy on test: 0.467
Timestamp:  13:29:02 	Val epoch 40 
	Model: resnet 
	Loss Train: 0.226 ,
	Loss Test: 1.64 ,
	Accuracy on test: 0.529
Timestamp:  13:29:38 	Val epoch 45 
	Model: resnet 
	Loss Train: 0.18

Timestamp:  14:28:21 	Val epoch 55 
	Model: densenet 
	Loss Train: 0.22 ,
	Loss Test: 1.22 ,
	Accuracy on test: 0.641
Timestamp:  14:29:38 	Val epoch 60 
	Model: densenet 
	Loss Train: 0.183 ,
	Loss Test: 1.19 ,
	Accuracy on test: 0.658
Timestamp:  14:30:55 	Val epoch 65 
	Model: densenet 
	Loss Train: 0.138 ,
	Loss Test: 1.08 ,
	Accuracy on test: 0.691
Timestamp:  14:32:12 	Val epoch 70 
	Model: densenet 
	Loss Train: 0.111 ,
	Loss Test: 1.15 ,
	Accuracy on test: 0.685
Timestamp:  14:33:31 	Val epoch 75 
	Model: densenet 
	Loss Train: 0.0764 ,
	Loss Test: 1.04 ,
	Accuracy on test: 0.717
Timestamp:  14:34:50 	Val epoch 80 
	Model: densenet 
	Loss Train: 0.0392 ,
	Loss Test: 0.954 ,
	Accuracy on test: 0.747
Timestamp:  14:36:07 	Val epoch 85 
	Model: densenet 
	Loss Train: 0.0352 ,
	Loss Test: 0.914 ,
	Accuracy on test: 0.754
Timestamp:  14:37:25 	Val epoch 90 
	Model: densenet 
	Loss Train: 0.0361 ,
	Loss Test: 0.91 ,
	Accuracy on test: 0.759
Timestamp:  14:38:42 	Val epoch 95 
	Model:

Timestamp:  15:12:17 	Val epoch 5 
	Model: densenet 
	Loss Train: 0.258 ,
	Loss Test: 1.16 ,
	Accuracy on test: 0.681
Timestamp:  15:13:05 	Val epoch 10 
	Model: densenet 
	Loss Train: 0.251 ,
	Loss Test: 1.11 ,
	Accuracy on test: 0.684
Timestamp:  15:13:54 	Val epoch 15 
	Model: densenet 
	Loss Train: 0.255 ,
	Loss Test: 1.32 ,
	Accuracy on test: 0.637
Timestamp:  15:14:42 	Val epoch 20 
	Model: densenet 
	Loss Train: 0.251 ,
	Loss Test: 1.4 ,
	Accuracy on test: 0.621
Timestamp:  15:15:31 	Val epoch 25 
	Model: densenet 
	Loss Train: 0.218 ,
	Loss Test: 1.24 ,
	Accuracy on test: 0.642
Timestamp:  15:16:21 	Val epoch 30 
	Model: densenet 
	Loss Train: 0.257 ,
	Loss Test: 1.17 ,
	Accuracy on test: 0.653
Timestamp:  15:17:10 	Val epoch 35 
	Model: densenet 
	Loss Train: 0.219 ,
	Loss Test: 1.26 ,
	Accuracy on test: 0.641
Timestamp:  15:17:59 	Val epoch 40 
	Model: densenet 
	Loss Train: 0.173 ,
	Loss Test: 1.22 ,
	Accuracy on test: 0.655
Timestamp:  15:18:47 	Val epoch 45 
	Model: densen

Timestamp:  15:39:58 	Val epoch 55 
	Model: vgg 
	Loss Train: 0.0176 ,
	Loss Test: 1.19 ,
	Accuracy on test: 0.63
Timestamp:  15:40:08 	Val epoch 60 
	Model: vgg 
	Loss Train: 0.0174 ,
	Loss Test: 1.2 ,
	Accuracy on test: 0.626
Timestamp:  15:40:17 	Val epoch 65 
	Model: vgg 
	Loss Train: 0.017 ,
	Loss Test: 1.19 ,
	Accuracy on test: 0.637
Timestamp:  15:40:27 	Val epoch 70 
	Model: vgg 
	Loss Train: 0.0172 ,
	Loss Test: 1.19 ,
	Accuracy on test: 0.631
Timestamp:  15:40:37 	Val epoch 75 
	Model: vgg 
	Loss Train: 0.0183 ,
	Loss Test: 1.18 ,
	Accuracy on test: 0.63
Timestamp:  15:40:46 	Val epoch 80 
	Model: vgg 
	Loss Train: 0.018 ,
	Loss Test: 1.16 ,
	Accuracy on test: 0.635
Timestamp:  15:40:56 	Val epoch 85 
	Model: vgg 
	Loss Train: 0.0176 ,
	Loss Test: 1.18 ,
	Accuracy on test: 0.635
Timestamp:  15:41:06 	Val epoch 90 
	Model: vgg 
	Loss Train: 0.0176 ,
	Loss Test: 1.15 ,
	Accuracy on test: 0.639
Timestamp:  15:41:15 	Val epoch 95 
	Model: vgg 
	Loss Train: 0.0175 ,
	Loss Test: 1.